In [2]:
# Install necessary packages
!pip install simpy
!pip install Topsis-Pulkit-102303800

# Import libraries
import simpy
import random
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [3]:
# 1. Define the Simulation (Call Center)
def call_center(env, num_agents, service_time, arrival_interval, wait_times):
    """Simulates a call center environment"""
    counter = simpy.Resource(env, capacity=num_agents)

    def customer(env, name, counter, service_time):
        arrival = env.now
        with counter.request() as req:
            yield req
            # Record wait time
            wait = env.now - arrival
            wait_times.append(wait)
            # Service
            yield env.timeout(random.expovariate(1.0 / service_time))

    def customer_generator(env, counter, service_time, arrival_interval):
        i = 0
        while True:
            yield env.timeout(random.expovariate(1.0 / arrival_interval))
            i += 1
            env.process(customer(env, f'Cust {i}', counter, service_time))

    env.process(customer_generator(env, counter, service_time, arrival_interval))
    env.run(until=480) # Run for 8 hours

# 2. Generate 1000 Simulations
print("Generating 1000 simulations... Please wait.")
data = []

for _ in range(1000):
    # Random Inputs
    num_agents = random.randint(2, 6)
    avg_service_time = random.uniform(5, 15)
    avg_arrival_interval = random.uniform(2, 6)

    # Run Sim
    env = simpy.Environment()
    wait_times = []
    call_center(env, num_agents, avg_service_time, avg_arrival_interval, wait_times)

    # Calculate Output (Target)
    avg_wait = np.mean(wait_times) if wait_times else 0
    data.append([num_agents, avg_service_time, avg_arrival_interval, avg_wait])

# 3. Save Data to CSV
df = pd.DataFrame(data, columns=['Agents', 'ServiceTime', 'ArrivalInterval', 'AvgWaitTime'])
df.to_csv("simulation_data.csv", index=False)
print("Success: 'simulation_data.csv' created.")
print(df.head())

Generating 1000 simulations... Please wait.
Success: 'simulation_data.csv' created.
   Agents  ServiceTime  ArrivalInterval  AvgWaitTime
0       5     5.796573         2.498328     0.165870
1       6    13.543669         4.212707     0.592843
2       6     7.637555         3.878722     0.001495
3       4     7.117076         5.562204     0.143936
4       3     6.505848         2.221345    25.867529


In [4]:
# Load Data
df = pd.read_csv("simulation_data.csv")
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Models
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=50, random_state=42),
    "GradientBoost": GradientBoostingRegressor(random_state=42),
    "KNeighbors": KNeighborsRegressor(),
    "DecisionTree": DecisionTreeRegressor(random_state=42)
}

# Evaluate Models
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    # Metrics
    r2 = r2_score(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)

    results.append([name, r2, rmse, mae])

# Save Performance to CSV
# Columns: Model Name, R2 (Maximize), RMSE (Minimize), MAE (Minimize)
perf_df = pd.DataFrame(results, columns=["Model", "R2", "RMSE", "MAE"])
perf_df.to_csv("model_performance.csv", index=False)

print("Success: 'model_performance.csv' created.")
print(perf_df)

Success: 'model_performance.csv' created.
              Model        R2       RMSE        MAE
0  LinearRegression  0.492446  21.507544  17.386867
1      RandomForest  0.810730  13.133791   6.603495
2     GradientBoost  0.804792  13.338245   7.398485
3        KNeighbors  0.804143  13.360402   6.610572
4      DecisionTree  0.714114  16.141569   7.650945


In [5]:
# Run your specific TOPSIS package via Command Line
# Syntax: topsis <InputDataFile> <Weights> <Impacts> <ResultFileName>

!topsis model_performance.csv "1,1,1" "+,-,-" topsis_result.csv

TOPSIS calculation complete. Output saved.


In [6]:
if os.path.exists("topsis_result.csv"):
    final_df = pd.read_csv("topsis_result.csv")

    # Sort by Rank (Rank 1 is best)
    # Note: Depending on your package version, the column might be 'Rank' or 'Topsis Score'
    # We will display the whole table.
    print("\n--- Final TOPSIS Ranking ---")
    print(final_df)

    # Identify the Winner
    # Assuming 'Rank' is the last column
    best_model_row = final_df.loc[final_df['Rank'] == 1]
    print(f"\n The Best Model according to TOPSIS is:\n {best_model_row.iloc[0,0]}")
else:
    print("Error: topsis_result.csv was not found. Check if the package ran correctly.")


--- Final TOPSIS Ranking ---
              Model        R2       RMSE        MAE  Topsis Score  Rank
0  LinearRegression  0.492446  21.507544  17.386867      0.000000     2
1      RandomForest  0.810730  13.133791   6.603495      1.000000     4
2     GradientBoost  0.804792  13.338245   7.398485      0.936948     3
3        KNeighbors  0.804143  13.360402   6.610572      0.986808     5
4      DecisionTree  0.714114  16.141569   7.650945      0.808649     1

 The Best Model according to TOPSIS is:
 DecisionTree
